In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import warnings
from datetime import datetime
import lightgbm as lgb

warnings.filterwarnings('ignore')

In [17]:
df0 = pd.read_pickle('train_10000_1_.pkl')
df1 = pd.read_pickle('train_10000_1.pkl')
df2 = pd.read_pickle('train_10000_2.pkl')

In [27]:

df = pd.concat([df1, df2.drop(['uin','label', 'kill_count', 'kill_time'], axis=1), 
                df0.drop(['uin','label', 'kill_count', 'kill_time'], axis=1)], axis=1)
df.dropna(axis=0, subset=['kill_time'], inplace=True)

In [28]:
y = df['label']
X_train = df.copy()
features = X_train.columns
features = features.drop(['uin','label', 'kill_count'])

In [29]:
X_test = pd.read_pickle('test.pkl')
X_test.head()

,uin,kill_time,pitch_r_range_max,pitch_r_range_min,pitch_r_range_mean,pitch_r_range_std,pitch_r_range_skew,yaw_r_range_max,yaw_r_range_min,yaw_r_range_mean,...,button_w2v_mean_0,button_w2v_mean_1,button_w2v_mean_2,button_w2v_mean_3,button_w2v_mean_4,button_w2v_mean_5,button_w2v_mean_6,button_w2v_mean_7,button_w2v_mean_8,button_w2v_mean_9
0,55f56b53f31dab28ca80aceb5a926945,16779046.0,0.178572,-0.178572,0.000093,0.092476,0.045378,0.698555,-0.694967,0.000711,...,1.231691,-2.203113,1.279010,3.939590,0.859231,1.912079,4.371671,2.263027,0.954478,4.633574
1,55f56b53f31dab28ca80aceb5a926945,16779048.0,0.165876,-0.165324,0.000073,0.064542,0.047977,0.720087,-0.726160,-0.000026,...,1.231178,-2.208704,1.266152,3.920979,0.858683,1.906994,4.347263,2.249064,0.952844,4.623341
2,55f56b53f31dab28ca80aceb5a926945,16779326.0,0.123372,-0.122682,0.000024,0.044547,0.023328,0.321404,-0.335480,0.000010,...,1.231691,-2.203113,1.279010,3.939590,0.859231,1.912079,4.371671,2.263027,0.954478,4.633574
3,55f56b53f31dab28ca80aceb5a926945,16779332.0,0.161322,-0.165048,0.000197,0.056243,-0.013476,0.856842,-0.860292,-0.000435,...,1.231691,-2.203113,1.279010,3.939590,0.859231,1.912079,4.371671,2.263027,0.954478,4.633574
4,4bbcb5adc84ba8d74df305253747f0e8,16797960.0,0.192246,-0.214800,0.000193,0.075507,-0.085947,2.752662,-2.688759,0.000237,...,1.223232,-2.174828,1.269668,3.895736,0.845891,1.886946,4.322258,2.212547,0.901127,4.570193


In [30]:
df_ = pd.read_csv('label.csv')
df_.head()

,uin,label,kill_count
0,0d75e2c05c4893bc3c2a97bbc6e34bb1,0.0,7
1,edca8de6a59bf769f39e167816721bd6,0.0,3
2,0ac54b353fb0546f81a6875f38f01915,0.0,12
3,04be3bdd0f6759215cbf3b6797a505eb,0.0,6
4,f4dd19d3b4d5d2ff6e3f84ed7818e980,0.0,39


In [31]:
black = pd.read_csv('训练集black_uin.txt', sep='\|', header=None)
white = pd.read_csv('训练集white_uin.txt', sep='\|', header=None)
black.columns = ['uin']
white.columns = ['uin']
black['label'] = 1
white['label'] = 0
label_map = pd.concat([black, white], axis=0)
label_map = dict(zip(label_map['uin'], label_map['label']))

In [61]:
test_uin = list(set(test['uin']))
count = 0
for uin in test_uin:
    if uin in label_map.keys():
        
        count += 1
count

942

In [32]:
# KF = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)
KF = GroupKFold(n_splits=10)
params = {
          'objective':'binary',
          'metric':'binary_error', 
          'learning_rate':0.05, 
          'subsample':0.8, 
          'subsample_freq':3, 
          'colsample_btree':0.8,
          'num_iterations': 10000, 
          'silent':True
}

oof_lgb = np.zeros(len(X_train))
predictions_lgb = np.zeros((len(X_test)))

# 五折交叉验证
for fold_, (trn_idx, val_idx) in enumerate(KF.split(X_train.values, y.values, X_train['uin'])):
    print("fold n°{}".format(fold_))
    print('trn_idx:',trn_idx)
    print('val_idx:',val_idx)
    trn_data = lgb.Dataset(X_train.iloc[trn_idx][features],label=y.iloc[trn_idx])    
    val_data = lgb.Dataset(X_train.iloc[val_idx][features],label=y.iloc[val_idx])
    num_round = 10000
    clf = lgb.train(
        params,
        trn_data,
        num_round,
        valid_sets = [trn_data, val_data],
        verbose_eval=500,
        early_stopping_rounds=200,  
        # categorical_feature=cat_cols,    
    )       
    oof_lgb[val_idx] = clf.predict(X_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions_lgb[:] += clf.predict(X_test[features], num_iteration=clf.best_iteration) 
print("AUC score: {}".format(roc_auc_score(y, oof_lgb)))
print("F1 score: {}".format(f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Precision score: {}".format(precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Recall score: {}".format(recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))

fold n°0
trn_idx: [     0      1      2 ... 121814 121815 121816]
val_idx: [   215    216    217 ... 121768 121769 121784]
Training until validation scores don't improve for 200 rounds
[500]	training's binary_error: 0.02273	valid_1's binary_error: 0.0897225
Early stopping, best iteration is:
[344]	training's binary_error: 0.033493	valid_1's binary_error: 0.0870957
fold n°1
trn_idx: [     0      1      2 ... 121814 121815 121816]
val_idx: [    67     68     69 ... 121792 121806 121807]
Training until validation scores don't improve for 200 rounds
[500]	training's binary_error: 0.0221918	valid_1's binary_error: 0.0995731
Early stopping, best iteration is:
[321]	training's binary_error: 0.0357915	valid_1's binary_error: 0.0953866
fold n°2
trn_idx: [     0      1      2 ... 121814 121815 121816]
val_idx: [    22     23     24 ... 121794 121795 121796]
Training until validation scores don't improve for 200 rounds
[500]	training's binary_error: 0.0229762	valid_1's binary_error: 0.0964538
Ear

In [33]:
X_test['predict'] = predictions_lgb / 10
X_test['max_prob'] = X_test.groupby('uin')['predict'].transform('max')
test_uin_df = X_test.drop_duplicates('uin')

In [34]:
sum([1 if i >= 0.5 else 0 for i in test_uin_df['max_prob']])

669

In [36]:
def post(x):
    if x['uin'] in label_map.keys():
        return label_map[x['uin']]
    elif x['max_prob'] >= 0.5:
        return 1
    else:
        return 0
test_uin_df['predict_label'] = test_uin_df.apply(post, axis=1)

In [37]:
submit = pd.read_csv('测试集玩家名单.txt', sep='\|', header=None)
submit.columns = ['uin']
submit = pd.merge(submit, test_uin_df[['predict_label', 'uin']], on='uin', how='left')
submit['predict_label'].to_excel('submit.xlsx', header=None, index=False)

In [38]:
sum(submit['predict_label'])

543

In [42]:
df['oof_predict'] = oof_lgb
df['max_prob'] = df.groupby('uin')['oof_predict'].transform('max')
uin_df = df.drop_duplicates('uin')
print("AUC score: {}".format(roc_auc_score(uin_df['label'], uin_df['max_prob'])))
print("F1 score: {}".format(f1_score(uin_df['label'], [1 if i >= 0.5 else 0 for i in uin_df['max_prob']])))
print("Precision score: {}".format(precision_score(uin_df['label'], [1 if i >= 0.5 else 0 for i in uin_df['max_prob']])))
print("Recall score: {}".format(recall_score(uin_df['label'], [1 if i >= 0.5 else 0 for i in uin_df['max_prob']])))
t = 0.5
P = precision_score(uin_df['label'], [1 if i >= t else 0 for i in uin_df['max_prob']])
R = recall_score(uin_df['label'], [1 if i >= t else 0 for i in uin_df['max_prob']])
oof_score = 4 * P * R / (P + 3 * R)
oof_score

AUC score: 0.8427096484098023
F1 score: 0.5567491826249416
Precision score: 0.6858457997698504
Recall score: 0.46855345911949686


0.6145913895333848

In [94]:
# df['oof_predict'] = oof_lgb
# df['max_prob'] = df.groupby('uin')['oof_predict'].transform('max')
# df['min_prob'] = df.groupby('uin')['oof_predict'].transform('min')
# df['mean_prob'] = df.groupby('uin')['oof_predict'].transform('mean')
# df['std_prob'] = df.groupby('uin')['oof_predict'].transform('std')

# X_train = df.drop_duplicates('uin')
# X_train = X_train[['max_prob', 'min_prob', 'mean_prob', 'std_prob', 'kill_count', 'label']]
# y = X_train['label']
# features = ['max_prob', 'min_prob', 'mean_prob', 'std_prob', 'kill_count']


# KF = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)

# params = {
#           'objective':'binary',
#           'metric':'binary_error', 
#           'learning_rate':0.05, 
#           'subsample':0.8, 
#           'subsample_freq':3, 
#           'colsample_btree':0.8,
#           'num_iterations': 10000, 
#           'silent':True
# }

# oof_lgb = np.zeros(len(X_train))
# # predictions_lgb = np.zeros((len(X_test)))

# # 五折交叉验证
# for fold_, (trn_idx, val_idx) in enumerate(KF.split(X_train.values, y.values)):
#     print("fold n°{}".format(fold_))
#     print('trn_idx:',trn_idx)
#     print('val_idx:',val_idx)
#     trn_data = lgb.Dataset(X_train.iloc[trn_idx][features],label=y.iloc[trn_idx])    
#     val_data = lgb.Dataset(X_train.iloc[val_idx][features],label=y.iloc[val_idx])
#     print(sum(y.iloc[val_idx]))
#     num_round = 10000
#     clf = lgb.train(
#         params,
#         trn_data,
#         num_round,
#         valid_sets = [trn_data, val_data],
#         verbose_eval=500,
#         early_stopping_rounds=200,  
#         # categorical_feature=cat_cols,    
#     )       
#     oof_lgb[val_idx] = clf.predict(X_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
#     # predictions_lgb[:] += clf.predict(X_test[features], num_iteration=clf.best_iteration) 
# print("AUC score: {}".format(roc_auc_score(y, oof_lgb)))
# print("F1 score: {}".format(f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
# print("Precision score: {}".format(precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
# print("Recall score: {}".format(recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))

fold n°0
trn_idx: [   0    1    2 ... 9996 9997 9999]
val_idx: [   3   15   23 ... 9985 9989 9998]
254.0
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[61]	training's binary_error: 0.086125	valid_1's binary_error: 0.0925
fold n°1
trn_idx: [   2    3    4 ... 9997 9998 9999]
val_idx: [   0    1    6 ... 9982 9983 9995]
254.0
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[95]	training's binary_error: 0.082	valid_1's binary_error: 0.0935
fold n°2
trn_idx: [   0    1    3 ... 9995 9997 9998]
val_idx: [   2    5   12 ... 9994 9996 9999]
254.0
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[85]	training's binary_error: 0.087	valid_1's binary_error: 0.0855
fold n°3
trn_idx: [   0    1    2 ... 9997 9998 9999]
val_idx: [   9   10   11 ... 9962 9964 9988]
255.0
Training until validation scores don't improve for 200 rounds
Early stopping, best it